In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [4]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3b
  atmpro = trp
  band = 3b
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.55, 1.5, 1.66)]
  wgt = [(0.8, 0.8, 0.8, 0.6, 0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.000087                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.000062
0.000750      2.0  0.000094                  2.0 -0.000049
0.001052      3.0  0.000111                  3.0  0.000128
0.001476      4.0  0.000129                  4.0  0.000156
0.002070      5.0  0.000148                  5.0  0.000171
0.002904      6.0  0.000167                  6.0  0.000186
0.004074      7.0  0.000187                  7.0  0.000202
0.005714      8.0  0.000208                  8.0  0.000217
0.008015      9.0  0.000230                  9.0  0.000233
0.011243     10.0  0.000252                 10.0  0.000250
0.015771     11.0  0.000274                 11.0  0.000267
0.022122     12.0  0.000298                 12.0  0.000285
0.031031     13.0  0.000322                 13.0  0.000303
0.043528     14.0  0.000349                 14.0  0.000309
0.061057     15.0  0.000432                 15.0  0.000358
0.085645     16.0  0.000701                 16.0  0.000570
0.120136     17.0  0.001212                 17.0  0.000959
0.168516     18.0  0.001936                 18.0  0.001487
0.236378     19.0  0.002923                 19.0  0.002199
0.331549     20.0  0.004242                 20.0  0.003144
0.465100     21.0  0.005978                 21.0  0.004399
0.652400     22.0  0.008215                 22.0  0.006119
0.915100     23.0  0.010369                 23.0  0.007801
1.283650     24.0  0.010399                 24.0  0.007920
1.800600     25.0  0.008443                 25.0  0.006599
2.525700     26.0  0.006398                 26.0  0.005166
3.542800     27.0  0.004728                 27.0  0.003948
4.969550     28.0  0.003453                 28.0  0.002982
6.970850     29.0  0.002534                 29.0  0.002251
9.778100     30.0  0.001871                 30.0  0.001695
13.715850    31.0  0.001366                 31.0  0.001250
19.239350    32.0  0.000981                 32.0  0.000899
26.987250    33.0  0.000730                 33.0  0.000660
37.855300    34.0  0.000576                 34.0  0.000507
53.100050    35.0  0.000447                 35.0  0.000384
73.887500    36.0  0.000335                 36.0  0.000282
97.662500    37.0  0.000279                 37.0  0.000231
121.437500   38.0  0.000263                 38.0  0.000215
145.212500   39.0  0.000248                 39.0  0.000200
168.987500   40.0  0.000269                 40.0  0.000219
192.762500   41.0  0.000584                 41.0  0.000477
216.537500   42.0  0.002350                 42.0  0.002017
240.312500   43.0  0.006827                 43.0  0.006292
264.087500   44.0  0.015031                 44.0  0.014830
287.862500   45.0  0.026836                 45.0  0.027340
311.637500   46.0  0.035972                 46.0  0.036447
335.412500   47.0  0.044634                 47.0  0.044793
359.187500   48.0  0.054799                 48.0  0.055616
382.962500   49.0  0.066441                 49.0  0.068167
406.737500   50.0  0.079516                 50.0  0.080550
430.512500   51.0  0.093934                 51.0  0.091915
454.287500   52.0  0.107622                 52.0  0.101499
478.062500   53.0  0.121814                 53.0  0.112260
501.837500   54.0  0.137498                 54.0  0.126221
525.612500   55.0  0.154630                 55.0  0.143317
549.387500   56.0  0.173164                 56.0  0.162724
573.162500   57.0  0.193229                 57.0  0.183039
596.937500   58.0  0.223813                 58.0  0.211662
620.712500   59.0  0.255233                 59.0  0.238950
644.487500   60.0  0.285041                 60.0  0.263725
668.262500   61.0  0.313058                 61.0  0.287991
692.037500   62.0  0.338756                 62.0  0.313320
715.812500   63.0  0.362089                 63.0  0.340742
739.587500   64.0  0.383002                 64.0  0.369430
76

# Fluxes by Layer

CRD                              CLIRAD             \
                         flug          fldg      fnetg       flug       fldg   
pressure     level                                                             
0.000000e+00 1     -38.664460  0.000000e+00 -38.664460        NaN        NaN   
1.000000e-08 1            NaN           NaN        NaN -38.768054   0.000182   
6.244000e-04 2     -38.664460  5.026141e-08 -38.664460 -38.768054   0.000182   
8.759000e-04 3     -38.664460  7.073615e-08 -38.664460 -38.768054   0.000182   
1.228600e-03 4     -38.664460  1.001601e-07 -38.664460 -38.768054   0.000182   
1.723400e-03 5     -38.664460  1.425242e-07 -38.664460 -38.768054   0.000182   
2.417400e-03 6     -38.664460  2.035053e-07 -38.664460 -38.768054   0.000182   
3.390900e-03 7     -38.664460  2.912839e-07 -38.664460 -38.768054   0.000182   
4.756500e-03 8     -38.664460  4.176285e-07 -38.664460 -38.768054   0.000182   
6.672000e-03 9     -38.664461  5.994596e-07 -38.664460 -38.768054   0.000182   
9.358900e-03 10    -38.664461  8.611365e-07 -38.664460 -38.768054   0.000182   
1.312780e-02 11    -38.664461  1.237692e-06 -38.664460 -38.768054   0.000183   
1.841450e-02 12    -38.664461  1.779531e-06 -38.664460 -38.768054   0.000183   
2.583020e-02 13    -38.664462  2.559154e-06 -38.664459 -38.768055   0.000184   
3.623230e-02 14    -38.664463  3.680864e-06 -38.664459 -38.768055   0.000184   
5.082340e-02 15    -38.664464  5.297334e-06 -38.664458 -38.768056   0.000186   
7.129060e-02 16    -38.664465  7.747969e-06 -38.664457 -38.768057   0.000187   
1.000000e-01 17    -38.664467  1.197590e-05 -38.664455 -38.768058   0.000190   
1.402710e-01 18    -38.664469  1.987057e-05 -38.664449 -38.768059   0.000196   
1.967600e-01 19    -38.664471  3.458318e-05 -38.664436 -38.768059   0.000206   
2.759970e-01 20    -38.664470  6.179104e-05 -38.664409 -38.768058   0.000225   
3.871000e-01 21    -38.664464  1.116185e-04 -38.664353 -38.768051   0.000260   
5.431000e-01 22    -38.664444  2.020829e-04 -38.664242 -38.768033   0.000323   
7.617000e-01 23    -38.664394  3.639471e-04 -38.664030 -38.767991   0.000440   
1.068500e+00 24    -38.664288  6.349934e-04 -38.663653 -38.767908   0.000641   
1.498800e+00 25    -38.664130  1.007474e-03 -38.663123 -38.767789   0.000926   
2.102400e+00 26    -38.663953  1.433750e-03 -38.662519 -38.767660   0.001268   
2.949000e+00 27    -38.663772  1.894087e-03 -38.661877 -38.767533   0.001659   
4.136600e+00 28    -38.663595  2.383081e-03 -38.661212 -38.767420   0.002101   
5.802500e+00 29    -38.663433  2.902003e-03 -38.660531 -38.767329   0.002599   
8.139200e+00 30    -38.663292  3.462110e-03 -38.659829 -38.767272   0.003165   
1.141700e+01 31    -38.663184  4.080672e-03 -38.659103 -38.767263   0.003814   
1.601470e+01 32    -38.663133  4.773987e-03 -38.658359 -38.767324   0.004556   
2.246400e+01 33    -38.663175  5.565006e-03 -38.657610 -38.767488   0.005407   
3.151050e+01 34    -38.663343  6.515893e-03 -38.656828 -38.767788   0.006415   
4.420010e+01 35    -38.663686  7.724519e-03 -38.655961 -38.768273   0.007662   
6.200000e+01 36    -38.664296  9.278571e-03 -38.655018 -38.769029   0.009227   
8.577500e+01 37    -38.665271  1.119760e-02 -38.654073 -38.770133   0.011126   
1.095500e+02 38    -38.666357  1.306984e-02 -38.653287 -38.771305   0.012949   
1.333250e+02 39    -38.667522  1.497463e-02 -38.652547 -38.772521   0.014771   
1.571000e+02 40    -38.668757  1.690719e-02 -38.651850 -38.773769   0.016583   
1.808750e+02 41    -38.670099  1.900851e-02 -38.651091 -38.775099   0.018530   
2.046500e+02 42    -38.672164  2.271600e-02 -38.649448 -38.777133   0.021908   
2.284250e+02 43    -38.677404  3.457607e-02 -38.642828 -38.782412   0.032868   
2.522000e+02 44    -38.688480  6.487949e-02 -38.623601 -38.793754   0.061932   
2.759750e+02 45    -38.708999  1.277301e-01 -38.581269 -38.814928   0.124876   
2.997500e+02 46    -38.743558  2.378681e-01 -38.505690 -38.850661   0.237616   
3.235250e+02 47    -38.789

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')